In [1]:
import src.view.pnl_view as pnlview
from multiprocessing import Pool, Process, Manager, cpu_count
from src.core.logger import Logger
from src.core.stopwatch import Stopwatch
from src.core.stopwatch_logger import StopwatchLogger
from src.synthetic_book_merger import SyntheticBookMerger
from src.synthetic_security_loader import SyntheticSecurityLoader
from src.sniper_opp_loader import SniperOppLoader

from src.view import data_view as dview
from src.view.opp_view import OppView, to_opp_view, display_opp_view
from src.view.cell_view import CellView
from src.view.color import Color, Colors, WebColors
from src.pcap_location_params import PCapLocationParams
from src.core.postgres_connection import PostgresConnection
from src.view.cell_view import CellView
from src.market_date import *

import pandas as pd
import numpy as np
import sqlite3
import time
import matplotlib.pyplot as plt
import os.path

# pandarallel.initialize(progress_bar=True, use_memory_fs=True)
pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220
pd.options.display.max_rows = 263

In [12]:
import importlib
import src.view.opp_view
import src.view.data_view as dview
import src.core.sort_utils as sort_utils
import src.sniper_opp_loader
import src.view.edge_summary_view
import src.synthetic_security_loader
import src.synthetic_book_merger
importlib.reload(module=src.view.opp_view)
importlib.reload(module=dview)
importlib.reload(module=src.view.edge_summary_view)
importlib.reload(module=src.synthetic_security_loader)
importlib.reload(module=src.synthetic_book_merger)
importlib.reload(module=src.sniper_opp_loader)
from src.view.opp_view import OppView
from src.view.edge_summary_view import get_edge_summary
from src.synthetic_security_loader import SyntheticSecurityLoader
from src.synthetic_book_merger import SyntheticBookMerger

In [3]:
sec = pd.read_feather(PCapLocationParams(340).feather_file_path(name='security', date='2021-06-02'))
sec['sym'] = sec['symbol'].astype('str')
sec.loc[sec['sym'].str.match('(ZSQ1-ZSU1|ZSQ1-ZSX1|ZSU1-ZSX1)')]
# sec.loc[sec['sym'].str.match('.*ZS.*')].sort_values('sym')

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id,sym
1032,19321,ZSQ1-ZSU1,2021-08-13 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70,ZSQ1-ZSU1
2133,49517,ZSQ1-ZSX1,2021-08-13 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70,ZSQ1-ZSX1
2353,20350,ZSU1-ZSX1,2021-09-14 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70,ZSU1-ZSX1


In [3]:
all_synth_secs = SyntheticSecurityLoader().result()
synth_secs = all_synth_secs.loc[all_synth_secs['symbol'].astype('str').str.match('.*ZS.*')].copy()
synth_secs['symbol'] = pd.Categorical(synth_secs['symbol'], categories=synth_secs['symbol'])
# synth_secs = synth_secs.set_index('symbol')
market_dates = {pid: set(get_feather_dates(340, name = f'synthetic_book_{pid}', month=[6])) for pid in range(1, 12)}
market_dates = list(set.intersection(*[set(v) for k, v in market_dates.items()]))
symbols=[
    '+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',
    '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',
    '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1']

opp_loader = src.sniper_opp_loader.SniperOppLoader(340, dates=market_dates, log_level=1, synth_secs=synth_secs)
opp_root = opp_loader.load_asset_opp_root(conn=PostgresConnection(host='titan').connect(), table_name='typed_sniper_opps')
opp = opp_loader.load()

[Info ] [2021-07-29 03:39:44.796366778]  |  Begin loading '' (16 symbols, 21 dates)
Reading sql query: select * from typed_sniper_opps where symbol!='' and run_id in (4, 3, 5, 2, 10, 14, 1, 6, 7, 8, 12, 13, 15, 9, 18, 16, 11, 20, 19, 17, 21, 41, 32, 35, 39, 40, 38, 34, 36, 37, 33, 23, 26, 24, 25, 22, 27, 42, 28, 29, 31, 30, 43, 46, 48, 50, 51, 58, 54, 59, 57, 47, 49, 45, 44, 61, 53, 60, 55, 52, 65, 66, 62, 64, 63, 56, 69, 74, 73, 77, 67, 70, 71, 72, 76, 79, 78, 68, 75, 80) and symbol in ('+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)', '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)', '+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2)', '+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1', '+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2)', '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1', '+ZSN1 -(ZSN1-ZSQ1) -ZSQ1', '+ZSN2 -(ZSN2-ZSX2) -ZSX2', '+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)', '+ZSN1 -(ZSN1-ZSX1) -ZSX1', '+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1', '+(ZSQ1-ZSU1) -(ZSQ1-ZSX1) +(ZSU1-ZSX1)', '+ZSQ1 -(ZSQ1-ZSU1) -(

In [4]:
synth_bk_date_groupby = opp_loader.synth_bk().groupby(level=[0])
# odg = opp.groupby(level=[0])
odg = opp.set_index(opp.index.set_names(['market_date', 'symbol', 'opp_start_eid'])).groupby(level=[0])
opp_w_bk_root = SyntheticBookMerger(odg, synth_bk_date_groupby).merge(parallel=True)

[Info ] [2021-07-29 03:39:52.650731755]  |  Begin loading synth books (all on 2021-06-15, 2021-06-06, 2021-06-30, 2021-06-13, and 17 others)
[Info ] [2021-07-29 03:39:52.789829702]  |  [2021-06-06] Processing 2021-06-06 polygon 1[Info ] [2021-07-29 03:39:52.789700244]  |  [2021-06-15] Processing 2021-06-15 polygon 1[Info ] [2021-07-29 03:39:52.790146817]  |  [2021-06-13] Processing 2021-06-13 polygon 1[Info ] [2021-07-29 03:39:52.790001148]  |  [2021-06-30] Processing 2021-06-30 polygon 1[Info ] [2021-07-29 03:39:52.790514863]  |  [2021-06-29] Processing 2021-06-29 polygon 1[Info ] [2021-07-29 03:39:52.790907799]  |  [2021-06-23] Processing 2021-06-23 polygon 1[Info ] [2021-07-29 03:39:52.790346146]  |  [2021-06-28] Processing 2021-06-28 polygon 1[Info ] [2021-07-29 03:39:52.791093830]  |  [2021-06-10] Processing 2021-06-10 polygon 1[Info ] [2021-07-29 03:39:52.791304861]  |  [2021-06-07] Processing 2021-06-07 polygon 1


[Info ] [2021-07-29 03:39:52.791904117]  |  [2021-06-17] Process

In [16]:
opp_w_bk_root.reset_index().to_feather('/home/john/spartan_store/pcaps/jupyter_cache/340/opp_w_bk_root_jun.feather')

In [11]:
opp_bk_tmp.sort_index()

t_time  id  bid_p_1  bid_q_1  bid_p_2  bid_q_2  bid_p_3  bid_q_3  ask_p_1  ask_q_1  ask_p_2  ask_q_2  ask_p_3  ask_q_3                    bk_dur                    fs_time                    ls_time                  bk_dur_lsn                  bk_dur_fsn                   opp_dur                 opp_dur_fsn                 opp_dur_lsn              opp_dur_thru            opp_dur_thru_fsn            opp_dur_thru_lsn  side  edge                     t_tod  is_origin                 lsn_delta
market_date symbol                                             opp_start_eid synth_bk_eid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
2021-06-01  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 91603         91603        2021-06-02 00:00:38.856343233   2     0.25        1     0.00        1    -0.25        2     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000555842 2021-06-02 00:00:38.857062 2021-06-02 00:00:38.857151 -1 days +23:59:59.999748075 -1 days +23:59:59.999837075           0 days 00:00:00 -1 days +23:59:59.999281233 -1 days +23:59:59.999192233 0 days 00:00:00.000555842 -1 days +23:59:59.999837075 -1 days +23:59:59.999748075  Sell  0.25 0 days 00:00:38.856343233       True 0 days 00:00:00.000807767
                                                                             91607        2021-06-02 00:00:38.856899075   2     0.25        1    -0.25        3    -0.50        3     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000040970 2021-06-02 00:00:38.857210 2021-06-02 00:00:38.857210 -1 days +23:59:59.999730045 -1 days +23:59:59.999730045 0 days 00:00:00.000555842 -1 days +23:59:59.999837075 -1 days +23:59:59.999748075 0 days 00:00:00.000596812 -1 days +23:59:59.999878045 -1 days +23:59:59.999789045  Sell  0.25 0 days 00:00:38.856899075      False 0 days 00:00:00.000807767
                                                                             91613        2021-06-02 00:00:38.856940045   2     0.00        1    -0.25        3    -0.50        2     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000074254 2021-06-02 00:00:38.857320 2021-06-02 00:00:38.857320 -1 days +23:59:59.999694299 -1 days +23:59:59.999694299 0 days 00:00:00.000596812 -1 days +23:59:59.999878045 -1 days +23:59:59.999789045 0 days 00:00:00.000671066 -1 days +23:59:59.999952299 -1 days +23:59:59.999863299   NaN  0.00 0 days 00:00:38.856940045      False 0 days 00:00:00.000807767
                                                                             91617        2021-06-02 00:00:38.857014299   2     0.00        1    -0.25        3    -0.50        1     2.50        2     2.75        1     3.00        1 0 days 00:00:00.000001780 2021-06-02 00:00:38.857374 2021-06-02 00:00:38.857374 -1 days +23:59:59.999642079 -1 days +23:59:59.999642079 0 days 00:00:00.000671066 -1 days +23:59:59.999952299 -1 days +23:59:59.999863299 0 days 00:00:00.000672846 -1 days +23:59:59.999954079 -1 days +23:59:59.999865079   NaN  0.00 0 days 00:00:38.857014299      False 0 days 00:00:00.000807767
                                                                             91619        2021-06-02 00:00:38.857016079   2     0.00        1    -0.25        3    -0.50        1     2.25        1     2.50        1     2.75        2 0 days 00:00:00.000007748 2021-06-02 00:00:38.857453 2021-06-02 00:00:38.857453 -1 days +23:59:59.999570827 -1 days +23:59:59.999570827 0 days 00:00:00.000672846 -1 days +23:59:59.999954079 -1 days +23:59:59.999865079 0 days 00:00:00.00068

In [14]:
src.view.opp_view.display_opp_view(src.view.opp_view.to_opp_view(opp_bk_tmp))

In [9]:
opp_bk_tmp = opp_w_bk_root.copy()
opp_bk_tmp['t_tod'] = opp_bk_tmp['t_time'] - opp_bk_tmp.index.get_level_values('market_date') - pd.Timedelta('1d')
opp_bk_tmp = dview.with_groupby_origin_col(opp_bk_tmp, levels=['market_date', 'symbol', 'opp_start_eid'])
ov_tmp = to_opp_view(opp_bk_tmp)
display_opp_view(ov_tmp)

In [12]:
opp_bk= opp_w_bk_root.copy()
opp_bk = pd.merge(
    opp_bk.reset_index('symbol').rename(columns={'symbol': 'sym'}),
    synth_secs, left_on='sym', right_on='symbol', how='left')\
    .set_index(opp_bk.index)

opp_bk_edge_summ = get_edge_summary(opp_bk, cpp=50.0, fee='member106j_fee')
es = opp_bk_edge_summ.copy()
edge_col_names = [(f'{e * .25:.2f}', dview.fmt_float_lambda(ndec=0)) for e in range(-2, 5)]
sdict = dict([(c, dview.fmt_float_lambda(ndec=2)) for c in ['net', 'net_cash', 'fill_edge', 'net_fill_cash']] + edge_col_names)
es = es.reset_index().set_index(['market_date', 'symbol'])
es.style.format(sdict)

In [13]:
es['month'] = pd.DatetimeIndex(es.index.get_level_values('market_date')).month
es_syms = es.index.get_level_values('symbol')
es['sym'] = es_syms

def get_monthly_pnl(x):
    return pd.DataFrame(data={str(m): [x.loc[x.name,m]['net_fill_cash'].sum() if (x.name, m) in x.index else 0.0] for m in range(4,8)})

es[['net_fill_cash', 'month', 'sym']].groupby(['sym', 'month']).sum().groupby(level='sym').apply(get_monthly_pnl)

,,4,5,6,7
sym,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),0,0.0,0.0,918.88,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),0,0.0,0.0,-41.78,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),0,0.0,0.0,807.66,0.0
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),0,0.0,0.0,-1210.74,0.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),0,0.0,0.0,-238.66,0.0
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,0,0.0,0.0,-149.90,0.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0,0.0,0.0,-193.80,0.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,0,0.0,0.0,-393.24,0.0
+ZSN1 -(ZSN1-ZSX1) -ZSX1,0,0.0,0.0,-226.60,0.0


In [4]:
synth_secs = SyntheticSecurityLoader(340).result().set_index('id')
# synth_secs['symbol'] = pd.Categorical(synth_secs['symbol'], categories=opp_bk_edge_summ.index.get_level_values('symbol').categories)
# synth_secs = synth_secs.set_index('symbol')
market_dates = {pid: set(get_feather_dates(340, name = f'synthetic_book_{pid}', month=[6])) for pid in range(1, 12)}
market_dates = list(set.intersection(*[set(v) for k, v in market_dates.items()]))
symbols=[
    '+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',
    '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',
    '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1']

opp_loader = SniperOppLoader(340, dates=market_dates, log_level=1)
opp_root = opp_loader.load_asset_opp_root('zs')
opp = opp_loader.load('zs')

[Info ] [2021-07-23 22:47:05.725102856]  |  Begin loading asset zs (11 symbols, 21 dates)
[Info ] [2021-07-23 22:47:09.123710480]  |  End loading asset zs (11 symbols, 21 dates).  Took 0 days 00:00:03.398484875


In [7]:
synth_bk_date_groupby = opp_loader.synth_bk().groupby(level=[0])
# odg = opp.groupby(level=[0])
odg = opp.set_index(opp.index.set_names(['market_date', 'symbol', 'opp_start_eid'])).groupby(level=[0])
opp_w_bk_root = SyntheticBookMerger(odg, synth_bk_date_groupby).merge(parallel=True)

[Info ] [2021-07-23 22:47:59.720817505]  |  Begin loading synth books (all on 2021-06-20, 2021-06-13, 2021-06-10, 2021-06-06, and 17 others)
[Info ] [2021-07-23 22:47:59.972217963]  |  [2021-06-20] Processing 2021-06-20 polygon 1[Info ] [2021-07-23 22:47:59.972354097]  |  [2021-06-13] Processing 2021-06-13 polygon 1[Info ] [2021-07-23 22:47:59.972520816]  |  [2021-06-10] Processing 2021-06-10 polygon 1[Info ] [2021-07-23 22:47:59.972672675]  |  [2021-06-06] Processing 2021-06-06 polygon 1[Info ] [2021-07-23 22:47:59.973345640]  |  [2021-06-14] Processing 2021-06-14 polygon 1[Info ] [2021-07-23 22:47:59.973554118]  |  [2021-06-29] Processing 2021-06-29 polygon 1


[Info ] [2021-07-23 22:47:59.973117480]  |  [2021-06-30] Processing 2021-06-30 polygon 1[Info ] [2021-07-23 22:47:59.973755774]  |  [2021-06-27] Processing 2021-06-27 polygon 1
[Info ] [2021-07-23 22:47:59.974168763]  |  [2021-06-15] Processing 2021-06-15 polygon 1

[Info ] [2021-07-23 22:47:59.975570075]  |  [2021-06-16] Proc

In [8]:
opp_bk_tmp = opp_w_bk_root.copy()
opp_bk_tmp['t_tod'] = opp_bk_tmp['t_time'] - opp_bk_tmp.index.get_level_values('market_date') - pd.Timedelta('1d')
opp_bk_tmp = dview.with_groupby_origin_col(opp_bk_tmp, levels=['market_date', 'symbol', 'opp_start_eid'])
ov_tmp = to_opp_view(opp_bk_tmp)
display_opp_view(ov_tmp)

In [12]:
opp_bk= opp_w_bk_root.copy()
opp_bk = pd.merge(
    opp_bk.reset_index('symbol').rename(columns={'symbol': 'sym'}),
    synth_secs, left_on='sym', right_on='symbol', how='left')\
    .set_index(opp_bk.index)

opp_bk_edge_summ = get_edge_summary(opp_bk, cpp=50.0, fee='member106j_fee')
es = opp_bk_edge_summ.copy()
edge_col_names = [(f'{e * .25:.2f}', dview.fmt_float_lambda(ndec=0)) for e in range(-2, 5)]
sdict = dict([(c, dview.fmt_float_lambda(ndec=2)) for c in ['net', 'net_cash', 'fill_edge', 'net_fill_cash']] + edge_col_names)
es = es.reset_index().set_index(['market_date', 'symbol'])
es.style.format(sdict)

In [13]:
es['month'] = pd.DatetimeIndex(es.index.get_level_values('market_date')).month
es_syms = es.index.get_level_values('symbol')
es['sym'] = es_syms

def get_monthly_pnl(x):
    return pd.DataFrame(data={str(m): [x.loc[x.name,m]['net_fill_cash'].sum() if (x.name, m) in x.index else 0.0] for m in range(4,8)})

es[['net_fill_cash', 'month', 'sym']].groupby(['sym', 'month']).sum().groupby(level='sym').apply(get_monthly_pnl)

,,4,5,6,7
sym,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),0,0.0,0.0,918.88,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),0,0.0,0.0,-41.78,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),0,0.0,0.0,807.66,0.0
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),0,0.0,0.0,-1210.74,0.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),0,0.0,0.0,-238.66,0.0
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,0,0.0,0.0,-149.90,0.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0,0.0,0.0,-193.80,0.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,0,0.0,0.0,-393.24,0.0
+ZSN1 -(ZSN1-ZSX1) -ZSX1,0,0.0,0.0,-226.60,0.0
